In [1]:
import pandas as pd, numpy as np

#, csv, requests, datetime, time, zipfile

In [2]:
path = 'Neighborhoods/' # select folder with data in it

#specify datatypes so FIPS numbers, etc. read in properly
dtypes = {'Geo_STATE': 'str', 'Geo_FIPS': 'str', 'Geo_COUNTY': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

# load acs data
colnames = ['Geo_STATE','Geo_FIPS','ACS16_5yr_B25003003','ACS16_5yr_B25003001','ACS16_5yr_B03002012','ACS16_5yr_B03002003',
            'ACS16_5yr_B03002004','ACS16_5yr_B03002001','ACS16_5yr_B03002012','ACS16_5yr_C17002001','ACS16_5yr_B25004001',
            'ACS16_5yr_C17002008','ACS16_5yr_C17002001','ACS16_5yr_C17002008','ACS16_5yr_C17002001','ACS16_5yr_B25024002',
            'ACS16_5yr_B25024003','ACS16_5yr_B25024004','ACS16_5yr_B25024005','ACS16_5yr_B25024006','ACS16_5yr_B25024007',
            'ACS16_5yr_B25024008','ACS16_5yr_B25024009','ACS16_5yr_B25024001','ACS16_5yr_B25034002','ACS16_5yr_B25034003',
            'ACS16_5yr_B25034004','ACS16_5yr_B25034010','ACS16_5yr_B25034011','ACS16_5yr_B25034001','ACS16_5yr_B15003022',
            'ACS16_5yr_B15003023','ACS16_5yr_B15003024','ACS16_5yr_B15003025','ACS16_5yr_B15003001','ACS16_5yr_B11005001',
            'ACS16_5yr_B11005002','ACS16_5yr_B03002005','ACS16_5yr_B03002006','ACS16_5yr_B03002007','Geo_COUNTY',
            'Density_OccHU_LandSqmi_2016','Geo_landsqmi','PopWtCent_longitude','PopWtCent_latitude',
            'ACS16_5yr_B25001001']

acs_data = pd.read_csv(path + 'ACS_Neighborhood_Raw.csv', usecols=(colnames),delimiter=',', dtype=dtypes)

acs_data.head()

,Geo_COUNTY,Geo_STATE,Geo_landsqmi,Geo_FIPS,PopWtCent_longitude,PopWtCent_latitude,Density_OccHU_LandSqmi_2016,ACS16_5yr_B25001001,ACS16_5yr_B25003001,ACS16_5yr_B25003003,...,ACS16_5yr_B03002012,ACS16_5yr_B11005001,ACS16_5yr_B11005002,ACS16_5yr_B15003001,ACS16_5yr_B15003022,ACS16_5yr_B15003023,ACS16_5yr_B15003024,ACS16_5yr_B15003025,ACS16_5yr_C17002001,ACS16_5yr_C17002008
0,047,19,162.087006,19047070500,-95.246545,41.914272,5.355149,928.0,868.0,233.0,...,133.0,868.0,271.0,1446.0,178.0,23.0,2.0,0.0,1977.0,1415.0
1,122,02,46.498001,02122000900,-151.567459,59.675132,18.538431,1173.0,862.0,146.0,...,58.0,862.0,267.0,1591.0,382.0,210.0,38.0,19.0,2227.0,1861.0
2,159,48,49.130001,48159950300,-95.238786,33.000882,20.537350,1376.0,1009.0,316.0,...,809.0,1009.0,309.0,1690.0,214.0,96.0,8.0,16.0,2813.0,1310.0
3,021,08,238.612000,08021974800,-105.980440,37.130390,5.699629,1783.0,1360.0,316.0,...,2492.0,1360.0,432.0,2385.0,302.0,90.0,11.0,2.0,3891.0,1791.0
4,061,37,128.440002,37061090100,-77.898112,35.118637,24.089067,3442.0,3094.0,775.0,...,4300.0,3094.0,1202.0,5762.0,342.0,125.0,40.0,0.0,9395.0,3333.0


In [3]:
#this file contains variables like rent, house values, unemployment
add_acs = pd.read_csv(path + 'Additional_ACS.csv', usecols=('Geo_FIPS', 'ACS16_5yr_B25064001', 'ACS16_5yr_B25077001',
                                                            'ACS16_5yr_B23025005','ACS16_5yr_B23025003'), 
                      delimiter=',', dtype=dtypes)
add_acs['FIPS_County'] = add_acs['Geo_FIPS'].str[:5]

#this file contains a cross tab between poverty status and race
colnames3 = ['Geo_FIPS', 'ACS16_5yr_B17020H001', 'ACS16_5yr_B17020H002', 'ACS16_5yr_B17020B001', 'ACS16_5yr_B17020B002', 
             'ACS16_5yr_B17020D001', 'ACS16_5yr_B17020D002', 'ACS16_5yr_B17020E001', 'ACS16_5yr_B17020E002',
             'ACS16_5yr_B17020I001', 'ACS16_5yr_B17020I002']
pov_race = pd.read_csv(path + 'Poverty_race.csv', usecols=(colnames3), delimiter=',', dtype=dtypes)

#this file contains household income by gross rent
colnames2 = ['Geo_FIPS','ACS16_5yr_B25074001','ACS16_5yr_B25074006','ACS16_5yr_B25074007','ACS16_5yr_B25074008',
            'ACS16_5yr_B25074009','ACS16_5yr_B25074015','ACS16_5yr_B25074016','ACS16_5yr_B25074017','ACS16_5yr_B25074018',
            'ACS16_5yr_B25074024','ACS16_5yr_B25074025','ACS16_5yr_B25074026','ACS16_5yr_B25074027','ACS16_5yr_B25074033',
            'ACS16_5yr_B25074034','ACS16_5yr_B25074035','ACS16_5yr_B25074036']
inc_rent = pd.read_csv(path + 'rent_income.csv', usecols=(colnames2), delimiter=',', dtype=dtypes)

#this file contains county data for unemployment rate, median rent, median house values
#Need to change variable names so they are differentiated from the census tract level variables
#colnames4 = ['Geo_FIPS','ACS16_5yr_B23025003','ACS16_5yr_B23025005','B25064001','B25077001']
colnames4 = [0, 57, 59, 69, 71]
newnames = ['FIPS_County','county_labor','county_unemp','county_med_rent','county_med_value']
county = pd.read_csv(path + 'county_data.csv', usecols=(colnames4), header=0, names=(newnames), delimiter=',', dtype=dtypes)

#this file contains jobs near tracts
employ = 'Employment/'
jobs = pd.read_csv(employ + 'tract nearby jobs master.csv', delimiter=',', dtype=dtypes)

county.head()

,FIPS_County,county_labor,county_unemp,county_med_rent,county_med_value
0,06001,862353,61327,1432,593500
1,06003,468,55,1109,329500
2,06005,14512,1677,1061,265900
3,06007,101952,10933,936,228500
4,06009,18323,1758,1120,259000


In [4]:
#this file contains county to CBSA crosswalk data
dtypes2 = {'CBSA Code': 'str', 'CBSA Title': 'str', 'County Name': 'str', 'FIPS State Code': 'str', 'FIPS_County': 'str'}
path_cbsa = 'OMB/'
cbsa_walk = pd.read_csv(path_cbsa + 'CBSA_walk.csv', header=0, delimiter=',', dtype=dtypes2)
cbsa_county = cbsa_walk.iloc[:,0:3].join(cbsa_walk.iloc[:,5:6])
cbsa_county['FIPS_County']='06' + cbsa_county['FIPS_County']

cbsa_county.head()
county = pd.merge(county, cbsa_county, on="FIPS_County", how="left")
county.head()

,FIPS_County,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,CBSA Title,County Name
0,06001,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County
1,06003,468,55,1109,329500,NaN,NaN,NaN
2,06005,14512,1677,1061,265900,NaN,NaN,NaN
3,06007,101952,10933,936,228500,17020,"Chico, CA",Butte County
4,06009,18323,1758,1120,259000,NaN,NaN,NaN


In [5]:
#merge together census tract datasets
merge1 = pd.merge(add_acs, pov_race, on="Geo_FIPS", how="outer")
merge2 = pd.merge(merge1, inc_rent, on="Geo_FIPS", how="outer")
merge3 = pd.merge(merge2, jobs, left_on = 'Geo_FIPS', right_on = 'h_tract', how="left")
merge4 = pd.merge(merge3, county, on = 'FIPS_County', how='left')

merge4.head()

,Geo_FIPS,ACS16_5yr_B23025003,ACS16_5yr_B23025005,ACS16_5yr_B25064001,ACS16_5yr_B25077001,FIPS_County,ACS16_5yr_B17020B001,ACS16_5yr_B17020B002,ACS16_5yr_B17020D001,ACS16_5yr_B17020D002,...,alljobs,ltbajobs,lt40kjobs,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,CBSA Title,County Name
0,06001400100,1643,75,3202.0,1074100.0,06001,85,0,499,10,...,763651.0,358749.0,301972.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County
1,06001400200,1270,41,1770.0,978900.0,06001,11,0,171,27,...,948962.0,453334.0,376760.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County
2,06001400300,3402,204,1208.0,912700.0,06001,603,188,591,83,...,986396.0,472646.0,397943.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County
3,06001400400,2678,114,1584.0,848900.0,06001,408,28,458,70,...,960234.0,459758.0,387879.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County
4,06001400500,2545,82,1438.0,683500.0,06001,944,126,140,7,...,985696.0,471892.0,399812.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",Alameda County


In [6]:
#left merge on this last one since Mark's dataset contains the entire US but we want to keep only CA tracts
master = pd.merge(merge4, acs_data, on='Geo_FIPS',how="left")

master.head()

,Geo_FIPS,ACS16_5yr_B23025003,ACS16_5yr_B23025005,ACS16_5yr_B25064001,ACS16_5yr_B25077001,FIPS_County,ACS16_5yr_B17020B001,ACS16_5yr_B17020B002,ACS16_5yr_B17020D001,ACS16_5yr_B17020D002,...,ACS16_5yr_B03002012,ACS16_5yr_B11005001,ACS16_5yr_B11005002,ACS16_5yr_B15003001,ACS16_5yr_B15003022,ACS16_5yr_B15003023,ACS16_5yr_B15003024,ACS16_5yr_B15003025,ACS16_5yr_C17002001,ACS16_5yr_C17002008
0,06001400100,1643,75,3202.0,1074100.0,06001,85,0,499,10,...,83.0,1292.0,241.0,2478.0,781.0,596.0,403.0,201.0,3011.0,2744.0
1,06001400200,1270,41,1770.0,978900.0,06001,11,0,171,27,...,211.0,813.0,184.0,1559.0,557.0,476.0,161.0,84.0,1952.0,1739.0
2,06001400300,3402,204,1208.0,912700.0,06001,603,188,591,83,...,386.0,2439.0,499.0,4124.0,1332.0,990.0,284.0,211.0,5153.0,4384.0
3,06001400400,2678,114,1584.0,848900.0,06001,408,28,458,70,...,266.0,1798.0,437.0,3303.0,1123.0,803.0,294.0,292.0,4158.0,3366.0
4,06001400500,2545,82,1438.0,683500.0,06001,944,126,140,7,...,486.0,1643.0,288.0,2943.0,1053.0,528.0,153.0,88.0,3733.0,2955.0


In [7]:
out = pd.DataFrame(master['Geo_FIPS']).join(master.loc[:,'alljobs':'County Name'])

out['renters'] = master['ACS16_5yr_B25003003']
out['occ_HU'] = master['ACS16_5yr_B25003001']

out['hispanic'] = master['ACS16_5yr_B03002012']
out['black'] = master['ACS16_5yr_B03002004']
out['white'] = master['ACS16_5yr_B03002003']
out['asian'] = master['ACS16_5yr_B03002006']
out['nhpi'] = master['ACS16_5yr_B03002007']
out['total_pop'] = master['ACS16_5yr_B03002001']

out['below200pov'] = master['ACS16_5yr_C17002001'] - master['ACS16_5yr_C17002008']
out['total_pov_status'] = master['ACS16_5yr_C17002001']

out['rent_occ_HU'] = master['ACS16_5yr_B25074001']
out['low_inc_rent_burden'] = (master['ACS16_5yr_B25074006'] + master['ACS16_5yr_B25074007'] + master['ACS16_5yr_B25074008'] + master['ACS16_5yr_B25074009'] + 
                              master['ACS16_5yr_B25074015'] + master['ACS16_5yr_B25074016'] + master['ACS16_5yr_B25074017'] + master['ACS16_5yr_B25074018'] + 
                              master['ACS16_5yr_B25074024'] + master['ACS16_5yr_B25074025'] + master['ACS16_5yr_B25074026'] + master['ACS16_5yr_B25074027'] + 
                              master['ACS16_5yr_B25074033'] + master['ACS16_5yr_B25074034'] + master['ACS16_5yr_B25074035'] + master['ACS16_5yr_B25074036'])

out['white_pov_tot'] = master['ACS16_5yr_B17020H001'] #NH white total
out['white_pov'] = master['ACS16_5yr_B17020H002'] #NH white poverty
out['black_pov_tot'] = master['ACS16_5yr_B17020B001'] #black total
out['black_pov'] = master['ACS16_5yr_B17020B002'] #black pov
out['asian_pov_tot'] = master['ACS16_5yr_B17020D001'] #asian total
out['asian_pov'] = master['ACS16_5yr_B17020D002'] #asian pov
out['nhpi_pov_tot'] = master['ACS16_5yr_B17020E001'] #NHPI total
out['nhpi_pov'] = master['ACS16_5yr_B17020E002'] #NHPI pov
out['hispanic_pov_tot'] = master['ACS16_5yr_B17020I001'] #hispanic total
out['hispanic_pov'] = master['ACS16_5yr_B17020I002'] #hispanic pov

out['sfdetach'] = master['ACS16_5yr_B25024002']
out['smallmf'] = master['ACS16_5yr_B25024004'] + master['ACS16_5yr_B25024005'] #2 units and 3-4 units
out['medmf'] = master['ACS16_5yr_B25024006'] + master['ACS16_5yr_B25024007'] #5-9 units and 10-19 units
out['bigmf'] = master['ACS16_5yr_B25024008'] + master['ACS16_5yr_B25024009'] #20-49 units and 50+ units
out['total_hu'] = master['ACS16_5yr_B25024001']
out['vacant_hu'] = master['ACS16_5yr_B25004001']
out['total_hu2'] = master['ACS16_5yr_B25001001']

out['since2000'] = master['ACS16_5yr_B25034002'] + master['ACS16_5yr_B25034003'] + master['ACS16_5yr_B25034004'] #2014 or later, 2010-2013, 2000-2009
out['before1950'] = master['ACS16_5yr_B25034010'] + master['ACS16_5yr_B25034011'] #1940-1949, 1939 or earlier
out['total_structure'] = master['ACS16_5yr_B25034001']

out['bach_degree'] = master['ACS16_5yr_B15003022'] + master['ACS16_5yr_B15003023'] + master['ACS16_5yr_B15003024'] + master['ACS16_5yr_B15003025']
out['pop_over_25'] = master['ACS16_5yr_B15003001']

out['hh_kids'] = master['ACS16_5yr_B11005002']
out['total_hh'] = master['ACS16_5yr_B11005001']

out['med_rent'] = master['ACS16_5yr_B25064001']
out['med_housevalue'] = master['ACS16_5yr_B25077001']

out['unemp_civ'] = master['ACS16_5yr_B23025005']
out['civ_labor_force'] = master['ACS16_5yr_B23025003']

out['area_sqmi'] = master['Geo_landsqmi']
out['HU_density'] = master['Density_OccHU_LandSqmi_2016']

print(out.columns)
out.head()


Index(['Geo_FIPS', 'alljobs', 'ltbajobs', 'lt40kjobs', 'county_labor',
       'county_unemp', 'county_med_rent', 'county_med_value', 'CBSA Code',
       'CBSA Title', 'County Name', 'renters', 'occ_HU', 'hispanic', 'black',
       'white', 'asian', 'nhpi', 'total_pop', 'below200pov',
       'total_pov_status', 'rent_occ_HU', 'low_inc_rent_burden',
       'white_pov_tot', 'white_pov', 'black_pov_tot', 'black_pov',
       'asian_pov_tot', 'asian_pov', 'nhpi_pov_tot', 'nhpi_pov',
       'hispanic_pov_tot', 'hispanic_pov', 'sfdetach', 'smallmf', 'medmf',
       'bigmf', 'total_hu', 'vacant_hu', 'total_hu2', 'since2000',
       'before1950', 'total_structure', 'bach_degree', 'pop_over_25',
       'hh_kids', 'total_hh', 'med_rent', 'med_housevalue', 'unemp_civ',
       'civ_labor_force', 'area_sqmi', 'HU_density'],
      dtype='object')


,Geo_FIPS,alljobs,ltbajobs,lt40kjobs,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,CBSA Title,...,bach_degree,pop_over_25,hh_kids,total_hh,med_rent,med_housevalue,unemp_civ,civ_labor_force,area_sqmi,HU_density
0,06001400100,763651.0,358749.0,301972.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",...,1981.0,2478.0,241.0,1292.0,3202.0,1074100.0,75,1643,2.657,486.262695
1,06001400200,948962.0,453334.0,376760.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",...,1278.0,1559.0,184.0,813.0,1770.0,978900.0,41,1270,0.230,3534.782471
2,06001400300,986396.0,472646.0,397943.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",...,2817.0,4124.0,499.0,2439.0,1208.0,912700.0,204,3402,0.427,5711.943848
3,06001400400,960234.0,459758.0,387879.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",...,2512.0,3303.0,437.0,1798.0,1584.0,848900.0,114,2678,0.272,6610.293945
4,06001400500,985696.0,471892.0,399812.0,862353,61327,1432,593500,41860,"San Francisco-Oakland-Hayward, CA",...,1822.0,2943.0,288.0,1643.0,1438.0,683500.0,82,2545,0.227,7237.885742


In [8]:
out.to_csv(path + 'tract_data.csv')

In [4]:
output = 'Output/' # select folder with data in it

#specify datatypes so FIPS numbers, etc. read in properly
dtypes = {'Geo_STATE': 'str', 'Geo_FIPS': 'str', 'Geo_COUNTY': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

out = pd.read_csv(output + 'tract_data.csv', delimiter=',', dtype=dtypes)

In [6]:
cols = ['renters', 'occ_HU', 'hispanic', 'black',
       'white', 'asian', 'nhpi', 'total_pop', 'below200pov',
       'total_pov_status', 'rent_occ_HU', 'low_inc_rent_burden',
       'white_pov_tot', 'white_pov', 'black_pov_tot', 'black_pov',
       'asian_pov_tot', 'asian_pov', 'nhpi_pov_tot', 'nhpi_pov',
       'hispanic_pov_tot', 'hispanic_pov', 'sfdetach', 'smallmf', 'medmf',
       'bigmf', 'total_hu', 'vacant_hu', 'total_hu2', 'since2000',
       'before1950', 'total_structure', 'bach_degree', 'pop_over_25',
       'hh_kids', 'total_hh','unemp_civ','civ_labor_force','area_sqmi']

avg_cols = ['alljobs','med_rent']

cbsa_tots=out[cols].groupby(out['CBSA Title']).sum()
cbsa_avgs=out[avg_cols].groupby(out['CBSA Title']).mean()


cbsa_stats = pd.DataFrame(cbsa_tots.iloc[:,7])

cbsa_stats['pct_rent']=cbsa_tots['renters'] / cbsa_tots['occ_HU']

cbsa_stats['pct_white']=cbsa_tots['white'] / cbsa_tots['total_pop']
cbsa_stats['pct_hispanic']=cbsa_tots['hispanic'] / cbsa_tots['total_pop']
cbsa_stats['pct_black']=cbsa_tots['black'] / cbsa_tots['total_pop']
cbsa_stats['pct_asian']=cbsa_tots['asian'] / cbsa_tots['total_pop']

cbsa_stats['pct_below200pov']=cbsa_tots['below200pov'] / cbsa_tots['total_pov_status']

cbsa_stats['hispanic_pov']=cbsa_tots['hispanic_pov'] / cbsa_tots['hispanic_pov_tot']
cbsa_stats['black_pov']=cbsa_tots['black_pov'] / cbsa_tots['black_pov_tot']
cbsa_stats['asian_pov']=cbsa_tots['asian_pov'] / cbsa_tots['asian_pov_tot']
cbsa_stats['white_pov']=cbsa_tots['white_pov'] / cbsa_tots['white_pov_tot']

cbsa_stats['pct_sfdetach']=cbsa_tots['sfdetach'] / cbsa_tots['total_hu']
cbsa_stats['pct_smallmf']=cbsa_tots['smallmf'] / cbsa_tots['total_hu']
cbsa_stats['pct_bigmf']=cbsa_tots['bigmf'] / cbsa_tots['total_hu']
cbsa_stats['pct_medmf']=cbsa_tots['medmf'] / cbsa_tots['total_hu']
cbsa_stats['pct_vacant']=cbsa_tots['vacant_hu'] / cbsa_tots['total_hu2']

cbsa_stats['pct_since2000']=cbsa_tots['since2000'] / cbsa_tots['total_structure']
cbsa_stats['pct_before1950']=cbsa_tots['before1950'] / cbsa_tots['total_structure']

cbsa_stats['pct_bach_degree']=cbsa_tots['bach_degree'] / cbsa_tots['pop_over_25']
cbsa_stats['pct_hh_kids']=cbsa_tots['hh_kids'] / cbsa_tots['total_hh']
cbsa_stats['pct_unemp_civ']=cbsa_tots['unemp_civ'] / cbsa_tots['civ_labor_force']
cbsa_stats['density']=cbsa_tots['total_pop'] / cbsa_tots['area_sqmi']

cbsa_stats['alljobs']=cbsa_avgs['alljobs']
cbsa_stats['med_rent']=cbsa_avgs['med_rent']

cbsa_stats.to_csv(output + 'cbsa_stats.csv')

cbsa_stats.head()

,total_pop,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,hispanic_pov,black_pov,asian_pov,...,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density,alljobs,med_rent
CBSA Title,,,,,,,,,,,,,,,,,,,,,
"Bakersfield, CA",871337.0,0.430850,0.360346,0.516157,0.051987,0.044579,0.476541,0.289656,0.349182,0.138010,...,0.060554,0.099402,0.215910,0.097655,0.156647,0.444303,0.119454,107.150283,101213.966887,1003.136986
"Chico, CA",223877.0,0.414642,0.734323,0.154116,0.014646,0.042421,0.429296,0.277212,0.370568,0.309698,...,0.069375,0.123143,0.139233,0.123727,0.261456,0.264312,0.107237,136.805414,28080.803922,988.333333
"Clearlake, CA",64076.0,0.368252,0.722595,0.188620,0.020741,0.013094,0.492120,0.351872,0.471756,0.405374,...,0.025342,0.264895,0.151545,0.090225,0.161568,0.266321,0.136766,50.997044,10090.533333,959.200000
"Crescent City, CA",27628.0,0.391295,0.630773,0.192775,0.022875,0.023527,0.460997,0.328247,0.447619,0.441536,...,0.027459,0.166255,0.158485,0.105245,0.149067,0.305623,0.104487,27.453042,5992.500000,765.142857
"El Centro, CA",178807.0,0.442271,0.119313,0.827037,0.023461,0.012455,0.503818,0.257607,0.289820,0.149450,...,0.094735,0.195164,0.265332,0.060661,0.139572,0.454891,0.170884,42.811596,25167.709677,837.766667
